In [2]:
import psycopg
import json
import folium
import numpy as np
import pandas as pd

In [3]:
conn = psycopg.connect(
        host="localhost",
        dbname="victoria_gtfs",
        user="username",
        password="password",
    )

In [4]:
route = '15'

cur = conn.execute(
    "SELECT DISTINCT ON (trips.shape_id) trips.shape_id,trips.trip_id, trips.direction_id, trips.trip_headsign FROM trips " 
    "JOIN routes ON trips.route_id=routes.route_id "
    "WHERE routes.route_short_name=%s ", (route,)
    
)
shapes = cur.fetchall()
shapes

[('1-15-147', 'aa5878544', 1, 'Esquimalt via Fort/Yates Exp'),
 ('1-15-150', 'aa5450463', 1, 'Esquimalt via Fort/Yates Exp'),
 ('1-15-151', 'aa6058786', 1, 'Esquimalt via Fort/Yates Exp'),
 ('1-15-154', 'aa6022634', 0, 'UVic via Foul Bay Exp'),
 ('1-15-155', 'aa5449629', 0, 'UVic via Foul Bay Exp'),
 ('1-15-156', 'aa5449936', 1, 'Esquimalt via Fort/Yates Exp')]

In [5]:
route_index = 4
shape = shapes[route_index][0]

cur = conn.execute(
    "SELECT ST_AsGeoJSON(shapes_aggregated.shape) "
    "FROM shapes_aggregated " 
    "WHERE shapes_aggregated.shape_id=%s ", (shape,)
)

line = cur.fetchall()
geojson = json.loads(line[0][0])

routeCoord = [v[::-1] for v in geojson["coordinates"]]

m = folium.Map(location=[51.05084309996687, -114.07251113843348], zoom_start=12)

folium.PolyLine(routeCoord, color="red", weight=5).add_to(m)

In [6]:
trip_id = shapes[route_index][1]
cur = conn.execute(
    """
    SELECT json_build_object(
        'type', 'FeatureCollection',
        'features', json_agg(ST_AsGeoJSON(t.*)::json)) FROM
    (SELECT stops.stop_loc, b.stop_sequence
    FROM (SELECT stop_times.stop_id, stop_times.stop_sequence FROM stop_times
    WHERE stop_times.trip_id=%s
    ORDER BY stop_times.stop_sequence) as b
    JOIN stops ON b.stop_id=stops.stop_id) as t
    """,
    (trip_id,))

stops = cur.fetchall()[0][0]

In [8]:
len(stops["features"])

31

In [6]:
coordinates = [i["geometry"]["coordinates"] for i in stops["features"]]
sequence = [i["properties"]["stop_sequence"] for i in stops["features"]]

for i, coord in enumerate(coordinates):
    folium.Marker(
        [coord[1], coord[0]],popup=f"<i>Stop Sequence: {sequence[i]}</i>"
    ).add_to(m)
m

In [7]:
def check_increasing(distances):
    myseries = pd.Series(distances)
    return myseries.is_unique and myseries.is_monotonic_increasing

In [8]:
# Alberta projection = 3402
# BC Projection = 3005

cur = conn.execute("""
    SELECT ST_LineLocatePoint(
        ST_SetSRID(shape_return.shape,4326), 
        stop_return.stop_loc::geometry) * 
        ST_Length(
            ST_Transform(
                ST_SetSRID(shape_return.shape,4326),
                3005
            )
        ) 
        FROM
    (
    SELECT stop_times.trip_id, trips.shape_id, stops.stop_loc, stop_times.stop_sequence, stop_times.stop_id FROM stop_times
    JOIN stops ON stop_times.stop_id=stops.stop_id
    JOIN trips ON stop_times.trip_id=trips.trip_id
    WHERE stop_times.trip_id=%s
    ORDER BY stop_times.stop_sequence
    ) AS stop_return
    JOIN (
    SELECT shapes_aggregated.shape, shapes_aggregated.shape_id
    FROM shapes_aggregated
    WHERE shapes_aggregated.shape_id=%s
    ) AS shape_return 
    ON stop_return.shape_id=shape_return.shape_id
    """,
    (shapes[route_index][1],shapes[route_index][0]))

distances = cur.fetchall()
distances = [i[0] for i in distances]

print(check_increasing(distances))
distances


True


[0.0,
 222.4332797843916,
 631.668211450481,
 853.3425243342361,
 1209.3418494368482,
 1432.2936655609644,
 1750.8932567724066,
 2072.0777086081275,
 2270.850920079976,
 2613.9547912119833,
 2828.3095891685903,
 3240.4436763003932,
 3533.084052368595,
 4096.4587969841305,
 4561.508527621063,
 4811.279035550974,
 5268.707599081394,
 5519.936238709723,
 5886.184222906559,
 6340.039339712699,
 6825.2098519439605,
 7447.8226155289485,
 8367.488142007143,
 8928.695629960905,
 9632.699335742465,
 10425.123792753082,
 10967.805612714048,
 11647.482541122188,
 12282.360529963908,
 12855.6045448555,
 13283.39031161014]

In [9]:
stop_distances = np.copy(distances)
end = stop_distances[-1]
for i in range(1,len(distances)-1):
    if (stop_distances[i-1]>stop_distances[i]):
        if(i<len(distances)/2):
            stop_distances[i-1]=end-stop_distances[i-1]
        else:
            stop_distances[i]=end-stop_distances[i]
        
print(check_increasing(stop_distances))


True


In [10]:
d = {'original': distances, 'modified': stop_distances, }
df = pd.DataFrame(d)
df

,original,modified
0,0.000000,0.000000
1,222.433280,222.433280
2,631.668211,631.668211
3,853.342524,853.342524
4,1209.341849,1209.341849
5,1432.293666,1432.293666
6,1750.893257,1750.893257
7,2072.077709,2072.077709
8,2270.850920,2270.850920
9,2613.954791,2613.954791


In [12]:
print(check_increasing(stop_distances))

True


In [163]:
# cur = conn.execute("""
#     WITH
#         shape_return AS (
#         SELECT 
#             shapes_aggregated.shape
#         FROM shapes_aggregated
#         WHERE shapes_aggregated.shape_id=%s
#         ),
#         stops_return AS (
#         SELECT 
#             stops.stop_loc
#         FROM (SELECT stop_times.stop_id, stop_times.stop_sequence FROM stop_times
#         WHERE stop_times.trip_id=%s
#         ORDER BY stop_times.stop_sequence) as b
#         JOIN stops ON b.stop_id=stops.stop_id limit 1
#         )
#     SELECT ST_LineLocatePoint(
#         (SELECT ST_SetSRID(shape_return.shape,4326) FROM shape_return), 
#         (SELECT stops_return.stop_loc::geometry FROM stops_return)
#         );
#     """,
#     (shapes[0][0],shapes[0][1]))

In [164]:
# cur = conn.execute("""
#     SELECT ST_LineLocatePoint(
#         ST_SetSRID(shape_return.shape,4326), 
#         stop_return.stop_loc::geometry) 
#         FROM
#     (
#     SELECT stop_times.trip_id, trips.shape_id, stops.stop_loc, stop_times.stop_sequence, stop_times.stop_id FROM stop_times
#     JOIN stops ON stop_times.stop_id=stops.stop_id
#     JOIN trips ON stop_times.trip_id=trips.trip_id
#     WHERE stop_times.trip_id=%s
#     ORDER BY stop_times.stop_sequence
#     ) AS stop_return
#     JOIN (
#     SELECT shapes_aggregated.shape, shapes_aggregated.shape_id
#     FROM shapes_aggregated
#     WHERE shapes_aggregated.shape_id=%s
#     ) AS shape_return 
#     ON stop_return.shape_id=shape_return.shape_id
#     """,
#     (shapes[0][1],shapes[0][0]))

# distances = cur.fetchall()
# distances = np.array([i[0] for i in distances])

# check_increasing(distances)
# distances

In [ ]:
# cur = conn.execute(
#     "SELECT ST_AsGeoJSON(shapes_aggregated.shape) "
#     "FROM (SELECT DISTINCT ON (trips.shape_id) trips.shape_id FROM trips " 
#     "JOIN routes ON trips.route_id=routes.route_id "
#     "WHERE routes.route_short_name=%s) AS b "
#     "JOIN shapes_aggregated ON b.shape_id = shapes_aggregated.shape_id ", (route,)
    
# )
